In [1]:
import io
import subprocess
import os
import glob
import sys
import pwd

import pandas as pd
import numpy as np
import scipy
import scipy.stats as stats
from sklearn.decomposition import PCA
import pickle
import re
import math
import plotnine as pn
import seaborn as sns
import glob
import subprocess
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from numpy import log
from collections import defaultdict,Counter
import re
import statsmodels
import warnings
import statsmodels.stats.multitest as smm
import json

from pathlib import Path
from midas import m
from datetime import datetime, date
import altair as alt

# gh specific
sys.path.append('/ghsfa/projects/pharma/shared_scripts/')
import bibp.functions as functions
import hgvslib.pHGVS as pHGVS  ## do a pull on the hgvslib git repo

# !pip install statannotations==0.4.0
# #!pip uninstall statannotations --yes
# from statannotations.Annotator import Annotator

warnings.filterwarnings("ignore")
pd.options.display.max_rows = None
pd.options.display.max_columns=None
pd.options.display.max_colwidth=None
work_dir="/ghsfa/projects/pharma/projects/sirius_pharma/hazhang_projects/AZN_SOW197_Infinity_01_Virus_CN_report_08152024"

# Generate ad-hoc Virus report -- Data processing

In [2]:
#Readin Infinity report of AZN_SOW197_Infinity_01
df_AZ_inf = pd.read_csv(f"{work_dir}/20240809_AstraZeneca_SOW197_30Samples_InfinityReport.csv", header = 1) 

print("Total sample size of AZN_SOW197_Infinity_01 batch is " + str(df_AZ_inf.shape[0]) + "\n") 
print("Total unique sample size of df_AZ_inf batch is " + str(df_AZ_inf.GHSampleID.nunique()) + "\n")
print("Total unique patient sample size of df_AZ_inf batch is " + str(df_AZ_inf.Patient_ID.nunique()) + "\n")

df_AZ_inf.head()

Total sample size of AZN_SOW197_Infinity_01 batch is 813

Total unique sample size of df_AZ_inf batch is 30

Total unique patient sample size of df_AZ_inf batch is 6



,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name
0,MATRIX_HNSCC,FB12680481,A1086304,B00713440,HN_Set-6,NaN,1,SUCCESS,NaN,SNV,NaN,ETV1,7,13971252.0,9,G226A,C>G,c.677G>C,NM_004956.5,42.92,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3242.0,6.33,NaN,NaN,Detected,0.001851,NaN,Not evaluable,Not Detected,19.21,3.5,0.0,06/11/2024,NaN,08/09/2024,HNSCC,Not Applicable,Not Applicable
1,MATRIX_HNSCC,FB12680481,A1086304,B00713440,HN_Set-6,NaN,1,SUCCESS,NaN,SNV,NaN,TNK2,3,195594512.0,12,R871H,C>T,c.2612G>A,NM_005781.5,41.89,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs763752077,NaN,NaN,NaN,NaN,2582.0,6.33,NaN,NaN,Detected,0.001851,NaN,Not evaluable,Not Detected,19.21,3.5,0.0,06/11/2024,NaN,08/09/2024,HNSCC,Not Applicable,Not Applicable
2,MATRIX_HNSCC,FB12680481,A1086304,B00713440,HN_Set-6,NaN,1,SUCCESS,NaN,SNV,NaN,TRAF3,14,103352540.0,6,T195T,C>T,c.585C>T,NM_003300.4,40.70,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs186117694,NaN,NaN,NaN,NaN,3125.0,6.33,NaN,NaN,Detected,0.001851,NaN,Not evaluable,Not Detected,19.21,3.5,0.0,06/11/2024,NaN,08/09/2024,HNSCC,Not Applicable,Not Applicable
3,MATRIX_HNSCC,FB12680481,A1086304,B00713440,HN_Set-6,NaN,1,SUCCESS,NaN,SNV,NaN,RECQL,12,21627845.0,11,M429V,T>C,c.1285A>G,NM_002907.4,40.09,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs368679142,NaN,NaN,NaN,NaN,2880.0,6.33,NaN,NaN,Detected,0.001851,NaN,Not evaluable,Not Detected,19.21,3.5,0.0,06/11/2024,NaN,08/09/2024,HNSCC,Not Applicable,Not Applicable
4,MATRIX_HNSCC,FB12680481,A1086304,B00713440,HN_Set-6,NaN,1,SUCCESS,NaN,SNV,NaN,ZNF217,20,52193081.0,4,K741R,T>C,c.2222A>G,NM_006526.3,39.92,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs569839543,NaN,NaN,NaN,NaN,2563.0,6.33,NaN,NaN,Detected,0.001851,NaN,Not evaluable,Not Detected,19.21,3.5,0.0,06/11/2024,NaN,08/09/2024,HNSCC,Not Applicable,Not Applicable


In [11]:
#Readin Infinity report of vfinal AZN_SOW197_Infinity_01 (downloaded from BART)
df_AZ_inf_vfinal = pd.read_csv(f"{work_dir}/AZN_SOW197_Infinity_01_BART_vFinal.csv", header = 3) 
df_AZ_inf_vfinal = df_AZ_inf_vfinal.drop_duplicates(subset = "run_sample_id")

print("Total sample size of AZN_SOW197_Infinity_01 batch of vfinal is " + str(df_AZ_inf_vfinal.shape[0]) + "\n") 
print("Total unique sample size of df_AZ_inf batch of vfinal is " + str(df_AZ_inf_vfinal.run_sample_id.nunique()) + "\n")
print("Total unique patient sample size of df_AZ_inf batch of vfinal is " + str(df_AZ_inf_vfinal.Patient_ID.nunique()) + "\n")

df_AZ_inf_vfinal.head()

Total sample size of AZN_SOW197_Infinity_01 batch of vfinal is 30

Total unique sample size of df_AZ_inf batch of vfinal is 30

Total unique patient sample size of df_AZ_inf batch of vfinal is 6



,index,Patient,run_sample_id,IDs,Patient_ID,Visit_name,Cancertype,GHRequestID,runid,GHDB_Table,Variant_type,methyl_tf,msre_model_name,methyl_score_BLADDER,methyl_score_BREAST,methyl_score_CRC,methyl_score_LUNG,cnv_type,gene,chrom,position,mut_aa,mut_nt,gene_b,pos_b,somatic_call,mutant_allele_status,msi_status,tmb_score,tmb_category,flags,run_sample_id_donor,has_replicate,n_contam_genes,Additional_comment,Recommended_manual_change,Manual_review_change,report_modules,data_dir,tb_html,oncotrack_pdf,fusion_bam,somatic_tsv,on_G360,on_OMNI,variant_module
0,1,1,B00713425,"B00713410, B00713425, B00713438, B00713460, B00713471",HN_Set-1,NaN,HNSCC,A1086330,240720_A01020_0846_BHJ32LDSXC,NaN,NaN,0.000000,AGGREGATED,-2.56320,-12.52307,-1.10846,-4.90495,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MSS/MSI-L,NaN,Not evaluable,NaN,NaN,NaN,NaN,NaN,NaN,no change,core;virus,/ghds/ivd/flowcentral/240720_A01020_0846_BHJ32LDSXC,NaN,B00713425.oncotrack.pdf,NaN,/ghds/ivd/flowcentral/240720_A01020_0846_BHJ32LDSXC/B00713425/B00713425.annotated.somatic.tsv,NaN,NaN,NaN
1,2,1,B00713438,"B00713410, B00713425, B00713438, B00713460, B00713471",HN_Set-1,NaN,HNSCC,A1086308,240720_A01020_0846_BHJ32LDSXC,sample_qc,Sample_level,0.002153,AGGREGATED,0.14070,-0.08758,3.89797,2.81637,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MSS/MSI-L,9.45,Low,Germline contamination,NaN,NaN,NaN,REVIEW contam_vscore 336.0 <= 50.0 genes,NaN,no change,core;virus,/ghds/ivd/flowcentral/240720_A01020_0846_BHJ32LDSXC,NaN,B00713438.oncotrack.pdf,NaN,/ghds/ivd/flowcentral/240720_A01020_0846_BHJ32LDSXC/B00713438/B00713438.annotated.somatic.tsv,NaN,NaN,NaN
4,5,1,B00713460,"B00713410, B00713425, B00713438, B00713460, B00713471",HN_Set-1,NaN,HNSCC,A1086313,240720_A01020_0846_BHJ32LDSXC,sample_qc,Sample_level,0.000000,AGGREGATED,-3.54091,-10.92306,-0.74960,-4.42506,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MSS/MSI-L,3.54,Low,Germline contamination,NaN,NaN,NaN,REVIEW contam_vscore 61.0 <= 50.0 genes,NaN,no change,core;virus,/ghds/ivd/flowcentral/240720_A01020_0846_BHJ32LDSXC,NaN,B00713460.oncotrack.pdf,NaN,/ghds/ivd/flowcentral/240720_A01020_0846_BHJ32LDSXC/B00713460/B00713460.annotated.somatic.tsv,NaN,NaN,NaN
7,8,1,B00713471,"B00713410, B00713425, B00713438, B00713460, B00713471",HN_Set-1,NaN,HNSCC,A1086314,240720_A01020_0846_BHJ32LDSXC,sample_qc,Sample_level,0.001311,AGGREGATED,2.76927,-7.30067,2.62177,0.58475,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MSS/MSI-L,5.91,Low,Germline contamination,NaN,NaN,NaN,REVIEW contam_vscore 222.0 <= 50.0 genes,NaN,no change,core;virus,/ghds/ivd/flowcentral/240720_A01020_0846_BHJ32LDSXC,NaN,B00713471.oncotrack.pdf,NaN,/ghds/ivd/flowcentral/240720_A01020_0846_BHJ32LDSXC/B00713471/B00713471.annotated.somatic.tsv,NaN,NaN,NaN
10,11,2,B00713412,"B00713412, B00713416, B00713419, B00713424, B00713474",HN_Set-2,NaN,HNSCC,A1086322,240720_A01020_0846_BHJ32LDSXC,NaN,NaN,0.000000,AGGREGATED,-3.77107,-2.47508,-1.97883,-5.96722,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MSS/MSI-L,NaN,Not evaluable,NaN,NaN,NaN,NaN,NaN,NaN,no change,core;virus,/ghds/ivd/flowcentral/240720_A01020_0846_BHJ32LDSXC,NaN,B00713412.oncotrack.pdf,NaN,/ghds/ivd/flowcentral/240720_A01020_0846_BHJ32LDSXC/B00713412/B00713412.annotated.somatic.tsv,NaN,NaN,NaN


## Retrieve fc_dir and cancer stage

In [ ]:
# # Prepare the "fc_dir" columns for the data
# df_AZ_inf_1 = df_AZ_inf.copy()

# # Define the path
# # Path to original BIP output dir
# fc_dir_all = "/ghsfa/ivd/flowcentral"
# fc_dir_omni = "/ghsfa/projects/omni_v2/flowcells"

# # Initialize a list to store rows that no BIP ouput found in either of the sources above
# rows_no_found = []

# # Initialize new columns in the DataFrame
# df_AZ_inf_1['fc_dir'] = ''

# # Iterate over the DataFrame rows
# for index, row in df_AZ_inf_1.iterrows():
#     runid = row["runid"]

#     # Find original folder
#     pattern_ori = f"{fc_dir_all}/*{runid}*"
#     folders = glob.glob(pattern_ori)
#     if folders:
#         df_AZ_inf_1.loc[index, 'fc_dir'] = folders[0]
#     else:
#         # Search in the alternative directory if no folder found in the first directory
#         pattern_omni = f"{fc_dir_omni}/*{runid}*"
#         folders_omni = glob.glob(pattern_omni)
#         if folders_omni:
#             df_AZ_inf_1.loc[index, 'fc_dir'] = folders_omni[0]
#         else:
#             # Print warning if no folder is found in either location
#             rows_no_found.append(row)
#             # print(f"No suitable folder found for {runid}")
            
# # Create a DataFrame with rows where folders were not found in any sources
# df_no_fc_dir = pd.DataFrame(rows_no_found)

# df_AZ_inf_1.head()


In [14]:
# Retrieve fc_dir
df_AZ_inf_vfinal_1 = df_AZ_inf_vfinal[["run_sample_id", "runid", "data_dir"]]
df_AZ_inf_vfinal_1 = df_AZ_inf_vfinal_1.rename(columns={"run_sample_id": "GHSampleID", "data_dir": "fc_dir"})

df_AZ_inf_1 = df_AZ_inf.merge(df_AZ_inf_vfinal_1[["GHSampleID", "runid", "fc_dir"]], on = "GHSampleID", how = "left")
df_AZ_inf_1.head()

,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name,runid,fc_dir
0,MATRIX_HNSCC,FB12680481,A1086304,B00713440,HN_Set-6,NaN,1,SUCCESS,NaN,SNV,NaN,ETV1,7,13971252.0,9,G226A,C>G,c.677G>C,NM_004956.5,42.92,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3242.0,6.33,NaN,NaN,Detected,0.001851,NaN,Not evaluable,Not Detected,19.21,3.5,0.0,06/11/2024,NaN,08/09/2024,HNSCC,Not Applicable,Not Applicable,240720_A01020_0846_BHJ32LDSXC,/ghds/ivd/flowcentral/240720_A01020_0846_BHJ32LDSXC
1,MATRIX_HNSCC,FB12680481,A1086304,B00713440,HN_Set-6,NaN,1,SUCCESS,NaN,SNV,NaN,TNK2,3,195594512.0,12,R871H,C>T,c.2612G>A,NM_005781.5,41.89,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs763752077,NaN,NaN,NaN,NaN,2582.0,6.33,NaN,NaN,Detected,0.001851,NaN,Not evaluable,Not Detected,19.21,3.5,0.0,06/11/2024,NaN,08/09/2024,HNSCC,Not Applicable,Not Applicable,240720_A01020_0846_BHJ32LDSXC,/ghds/ivd/flowcentral/240720_A01020_0846_BHJ32LDSXC
2,MATRIX_HNSCC,FB12680481,A1086304,B00713440,HN_Set-6,NaN,1,SUCCESS,NaN,SNV,NaN,TRAF3,14,103352540.0,6,T195T,C>T,c.585C>T,NM_003300.4,40.70,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs186117694,NaN,NaN,NaN,NaN,3125.0,6.33,NaN,NaN,Detected,0.001851,NaN,Not evaluable,Not Detected,19.21,3.5,0.0,06/11/2024,NaN,08/09/2024,HNSCC,Not Applicable,Not Applicable,240720_A01020_0846_BHJ32LDSXC,/ghds/ivd/flowcentral/240720_A01020_0846_BHJ32LDSXC
3,MATRIX_HNSCC,FB12680481,A1086304,B00713440,HN_Set-6,NaN,1,SUCCESS,NaN,SNV,NaN,RECQL,12,21627845.0,11,M429V,T>C,c.1285A>G,NM_002907.4,40.09,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs368679142,NaN,NaN,NaN,NaN,2880.0,6.33,NaN,NaN,Detected,0.001851,NaN,Not evaluable,Not Detected,19.21,3.5,0.0,06/11/2024,NaN,08/09/2024,HNSCC,Not Applicable,Not Applicable,240720_A01020_0846_BHJ32LDSXC,/ghds/ivd/flowcentral/240720_A01020_0846_BHJ32LDSXC
4,MATRIX_HNSCC,FB12680481,A1086304,B00713440,HN_Set-6,NaN,1,SUCCESS,NaN,SNV,NaN,ZNF217,20,52193081.0,4,K741R,T>C,c.2222A>G,NM_006526.3,39.92,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs569839543,NaN,NaN,NaN,NaN,2563.0,6.33,NaN,NaN,Detected,0.001851,NaN,Not evaluable,Not Detected,19.21,3.5,0.0,06/11/2024,NaN,08/09/2024,HNSCC,Not Applicable,Not Applicable,240720_A01020_0846_BHJ32LDSXC,/ghds/ivd/flowcentral/240720_A01020_0846_BHJ32LDSXC


In [27]:
# Retrieve cancer stage
# Merge cancer type information from /ghsfa/projects/pharma/datawarehouse/pdw/raw/latest/fact_accessions.tsv
df_cancer_type_fact = pd.read_table("/ghsfa/projects/pharma/datawarehouse/pdw/raw/latest/fact_accessions.tsv")
df_cancer_type_fact_1 = df_cancer_type_fact.copy()
df_AZ_inf_2 = df_AZ_inf_1.copy()

#merge the cancer stage information information from the fact_accessions.tsv
df_cancer_type_fact_1.rename(columns={'accn_id': 'GHRequestID'}, inplace=True)
df_AZ_inf_3 = df_AZ_inf_2.merge(df_cancer_type_fact_1[["GHRequestID", "cancer_category", "cancerother", "cancerstage", "cancertype"]], on = "GHRequestID", how = "left")

# Rename some columns
df_AZ_inf_3 = df_AZ_inf_3.rename(columns={"cancerstage": "Tumor_stage", "cfDNA_ng": "cfDNA_ng_yield"})
df_AZ_inf_3["ctDNA_detected"] = df_AZ_inf_3["Tumor_methylation_status"] 

print("Total sample size is " + str(df_AZ_inf_3.shape[0]) + "\n") 
print('Total NaN in cancer type of the column "Tumor_stage" is ' + str(df_AZ_inf_3.Tumor_stage.isnull().sum()) + "\n")
df_AZ_inf_3.head()

Total sample size is 813

Total NaN in cancer type of the column "Tumor_stage" is 813



,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng_yield,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name,runid,fc_dir,cancer_category,cancerother,Tumor_stage,cancertype,ctDNA_detected
0,MATRIX_HNSCC,FB12680481,A1086304,B00713440,HN_Set-6,NaN,1,SUCCESS,NaN,SNV,NaN,ETV1,7,13971252.0,9,G226A,C>G,c.677G>C,NM_004956.5,42.92,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3242.0,6.33,NaN,NaN,Detected,0.001851,NaN,Not evaluable,Not Detected,19.21,3.5,0.0,06/11/2024,NaN,08/09/2024,HNSCC,Not Applicable,Not Applicable,240720_A01020_0846_BHJ32LDSXC,/ghds/ivd/flowcentral/240720_A01020_0846_BHJ32LDSXC,Other,HNSCC,NaN,HNSCC,Detected
1,MATRIX_HNSCC,FB12680481,A1086304,B00713440,HN_Set-6,NaN,1,SUCCESS,NaN,SNV,NaN,TNK2,3,195594512.0,12,R871H,C>T,c.2612G>A,NM_005781.5,41.89,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs763752077,NaN,NaN,NaN,NaN,2582.0,6.33,NaN,NaN,Detected,0.001851,NaN,Not evaluable,Not Detected,19.21,3.5,0.0,06/11/2024,NaN,08/09/2024,HNSCC,Not Applicable,Not Applicable,240720_A01020_0846_BHJ32LDSXC,/ghds/ivd/flowcentral/240720_A01020_0846_BHJ32LDSXC,Other,HNSCC,NaN,HNSCC,Detected
2,MATRIX_HNSCC,FB12680481,A1086304,B00713440,HN_Set-6,NaN,1,SUCCESS,NaN,SNV,NaN,TRAF3,14,103352540.0,6,T195T,C>T,c.585C>T,NM_003300.4,40.70,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs186117694,NaN,NaN,NaN,NaN,3125.0,6.33,NaN,NaN,Detected,0.001851,NaN,Not evaluable,Not Detected,19.21,3.5,0.0,06/11/2024,NaN,08/09/2024,HNSCC,Not Applicable,Not Applicable,240720_A01020_0846_BHJ32LDSXC,/ghds/ivd/flowcentral/240720_A01020_0846_BHJ32LDSXC,Other,HNSCC,NaN,HNSCC,Detected
3,MATRIX_HNSCC,FB12680481,A1086304,B00713440,HN_Set-6,NaN,1,SUCCESS,NaN,SNV,NaN,RECQL,12,21627845.0,11,M429V,T>C,c.1285A>G,NM_002907.4,40.09,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs368679142,NaN,NaN,NaN,NaN,2880.0,6.33,NaN,NaN,Detected,0.001851,NaN,Not evaluable,Not Detected,19.21,3.5,0.0,06/11/2024,NaN,08/09/2024,HNSCC,Not Applicable,Not Applicable,240720_A01020_0846_BHJ32LDSXC,/ghds/ivd/flowcentral/240720_A01020_0846_BHJ32LDSXC,Other,HNSCC,NaN,HNSCC,Detected
4,MATRIX_HNSCC,FB12680481,A1086304,B00713440,HN_Set-6,NaN,1,SUCCESS,NaN,SNV,NaN,ZNF217,20,52193081.0,4,K741R,T>C,c.2222A>G,NM_006526.3,39.92,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs569839543,NaN,NaN,NaN,NaN,2563.0,6.33,NaN,NaN,Detected,0.001851,NaN,Not evaluable,Not Detected,19.21,3.5,0.0,06/11/2024,NaN,08/09/2024,HNSCC,Not Applicable,Not Applicable,240720_A01020_0846_BHJ32LDSXC,/ghds/ivd/flowcentral/240720_A01020_0846_BHJ32LDSXC,Other,HNSCC,NaN,HNSCC,Detected


In [46]:
# Pull the virus positive call information "Virus_strain" (column "virus") and "Virus_copy_number" (column "estimated_copy_number")
df_AZ_inf_vfinal_2 = df_AZ_inf_vfinal_1.copy()
df_AZ_inf_vfinal_2["Virus_strain"] = np.nan
df_AZ_inf_vfinal_2["Virus_copy_number"] = np.nan

# Prepare a list to collect data that needs to be appended
virus_pos_call = []
# Collect indices of rows that are expanded with virus data
indices_to_drop = []  

for index, row in df_AZ_inf_vfinal_2.iterrows():
    run_sample_id = row["GHSampleID"]
    fc_dir = row["fc_dir"]
    virus_file = f"{fc_dir}/{run_sample_id}/{run_sample_id}.virus_call.report.hdr.tsv" 

    try:
        df_virus_file = pd.read_table(virus_file)
        # select positive calls
        df_virus_file_pos = df_virus_file[df_virus_file["call"] == 1]

        if not df_virus_file_pos.empty:
            # Mark the original row to be dropped as it will be replaced
            indices_to_drop.append(index)  
            for _, virus_row in df_virus_file_pos.iterrows():
                new_row = row.copy()
                new_row["Virus_strain"] = virus_row['virus']
                new_row["Virus_copy_number"] = virus_row['estimated_copy_number']
                virus_pos_call.append(new_row)
                
    except FileNotFoundError:
        print(f"No file found for {run_sample_id}. Assigning np.nan to Virus_strain and Virus_copy_number.")
    except Exception as e:
        print(f"An error occurred with {run_sample_id}: {str(e)}")

# Drop the original rows that have been expanded with virus data
df_AZ_inf_vfinal_2.drop(indices_to_drop, inplace=True)

# Append virus positive call row
if virus_pos_call:
    df_AZ_inf_vfinal_2 = pd.concat([df_AZ_inf_vfinal_2, pd.DataFrame(virus_pos_call)], ignore_index=True)

df_AZ_inf_vfinal_2


,GHSampleID,runid,fc_dir,Virus_strain,Virus_copy_number
0,B00713412,240720_A01020_0846_BHJ32LDSXC,/ghds/ivd/flowcentral/240720_A01020_0846_BHJ32LDSXC,NaN,NaN
1,B00713416,240720_A01020_0846_BHJ32LDSXC,/ghds/ivd/flowcentral/240720_A01020_0846_BHJ32LDSXC,NaN,NaN
2,B00713419,240720_A01020_0846_BHJ32LDSXC,/ghds/ivd/flowcentral/240720_A01020_0846_BHJ32LDSXC,NaN,NaN
3,B00713424,240720_A01020_0846_BHJ32LDSXC,/ghds/ivd/flowcentral/240720_A01020_0846_BHJ32LDSXC,NaN,NaN
4,B00713474,240720_A01020_0846_BHJ32LDSXC,/ghds/ivd/flowcentral/240720_A01020_0846_BHJ32LDSXC,NaN,NaN
5,B00713420,240720_A01020_0846_BHJ32LDSXC,/ghds/ivd/flowcentral/240720_A01020_0846_BHJ32LDSXC,NaN,NaN
6,B00713467,240720_A01020_0846_BHJ32LDSXC,/ghds/ivd/flowcentral/240720_A01020_0846_BHJ32LDSXC,NaN,NaN
7,B00713414,240720_A01020_0846_BHJ32LDSXC,/ghds/ivd/flowcentral/240720_A01020_0846_BHJ32LDSXC,NaN,NaN
8,B00713415,240720_A01020_0846_BHJ32LDSXC,/ghds/ivd/flowcentral/240720_A01020_0846_BHJ32LDSXC,NaN,NaN
9,B00713440,240720_A01020_0846_BHJ32LDSXC,/ghds/ivd/flowcentral/240720_A01020_0846_BHJ32LDSXC,NaN,NaN


In [51]:
# Generate the final report 
df_AZ_inf_4 = df_AZ_inf_3.copy()

# Merge the virus info
df_AZ_inf_4 = df_AZ_inf_4.merge(df_AZ_inf_vfinal_2[["GHSampleID", "Virus_strain", "Virus_copy_number"]], on = "GHSampleID", how = "left")

# Subsetting the final report based on this template: https://docs.google.com/spreadsheets/d/14vCqb-qNID03qwuRSzULRjJ3dzx7-Db2/edit?gid=949784952#gid=949784952
df_AZ_inf_5 = df_AZ_inf_4[['Study_ID', 'Customer_SampleID', 'GHRequestID', 'GHSampleID', 'Patient_ID', "Cancertype", "Tumor_stage", "Visit_name", 'Sample_status', 'Sample_comment',  'ctDNA_detected', 'Tumor_methylation_status', 'Tumor_methylation_score', "Virus_strain", "Virus_copy_number", 'cfDNA_ng_yield', 'Plasma_ml_input',  'Plasma_ml_remaining', 'Received_date', 'Bloodcoll_date', 'Reported_date']]

# Drop duplicates
df_AZ_inf_final = df_AZ_inf_5.drop_duplicates(subset = "GHSampleID")
# Save the data
df_AZ_inf_final.to_csv(f"{work_dir}/df_AZ_inf_final.csv", index = False)

df_AZ_inf_final.head()

,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Cancertype,Tumor_stage,Visit_name,Sample_status,Sample_comment,ctDNA_detected,Tumor_methylation_status,Tumor_methylation_score,Virus_strain,Virus_copy_number,cfDNA_ng_yield,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date
0,MATRIX_HNSCC,FB12680481,A1086304,B00713440,HN_Set-6,HNSCC,NaN,NaN,SUCCESS,NaN,Detected,Detected,0.001851,NaN,NaN,19.21,3.5,0.0,06/11/2024,NaN,08/09/2024
64,MATRIX_HNSCC,FB12680088,A1086305,B00713441,HN_Set-5,HNSCC,NaN,NaN,SUCCESS,NaN,Detected,Detected,0.006236,NaN,NaN,18.16,3.0,0.0,06/11/2024,NaN,08/09/2024
138,MATRIX_HNSCC,FB12679892,A1086306,B00713443,HN_Set-5,HNSCC,NaN,NaN,SUCCESS,NaN,Detected,Detected,0.016373,NaN,NaN,25.46,4.0,0.0,06/11/2024,NaN,08/09/2024
197,MATRIX_HNSCC,FB12679870,A1086307,B00713437,HN_Set-4,HNSCC,NaN,NaN,SUCCESS,NaN,Not Detected,Not Detected,0.000000,NaN,NaN,13.61,4.0,0.0,06/11/2024,NaN,08/09/2024
214,MATRIX_HNSCC,FB12778707,A1086308,B00713438,HN_Set-1,HNSCC,NaN,NaN,SUCCESS,NaN,Detected,Detected,0.002153,HPV16,161.54,6.77,4.0,0.0,06/11/2024,NaN,08/09/2024


In [53]:
df_AZ_inf_final

,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Cancertype,Tumor_stage,Visit_name,Sample_status,Sample_comment,ctDNA_detected,Tumor_methylation_status,Tumor_methylation_score,Virus_strain,Virus_copy_number,cfDNA_ng_yield,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date
0,MATRIX_HNSCC,FB12680481,A1086304,B00713440,HN_Set-6,HNSCC,NaN,NaN,SUCCESS,NaN,Detected,Detected,0.001851,NaN,NaN,19.21,3.5,0.0,06/11/2024,NaN,08/09/2024
64,MATRIX_HNSCC,FB12680088,A1086305,B00713441,HN_Set-5,HNSCC,NaN,NaN,SUCCESS,NaN,Detected,Detected,0.006236,NaN,NaN,18.16,3.0,0.0,06/11/2024,NaN,08/09/2024
138,MATRIX_HNSCC,FB12679892,A1086306,B00713443,HN_Set-5,HNSCC,NaN,NaN,SUCCESS,NaN,Detected,Detected,0.016373,NaN,NaN,25.46,4.0,0.0,06/11/2024,NaN,08/09/2024
197,MATRIX_HNSCC,FB12679870,A1086307,B00713437,HN_Set-4,HNSCC,NaN,NaN,SUCCESS,NaN,Not Detected,Not Detected,0.000000,NaN,NaN,13.61,4.0,0.0,06/11/2024,NaN,08/09/2024
214,MATRIX_HNSCC,FB12778707,A1086308,B00713438,HN_Set-1,HNSCC,NaN,NaN,SUCCESS,NaN,Detected,Detected,0.002153,HPV16,161.54,6.77,4.0,0.0,06/11/2024,NaN,08/09/2024
243,MATRIX_HNSCC,FB12680381,A1086309,B00713439,HN_Set-4,HNSCC,NaN,NaN,SUCCESS,NaN,Not Detected,Not Detected,0.000000,HPV16,13.38,11.27,3.5,0.0,06/11/2024,NaN,08/09/2024
265,MATRIX_HNSCC,FB12679956,A1086310,B00713455,HN_Set-4,HNSCC,NaN,NaN,SUCCESS,NaN,Not Detected,Not Detected,0.000000,HPV16,92.81,10.92,3.5,0.0,06/11/2024,NaN,08/09/2024
292,MATRIX_HNSCC,FB12679876,A1086311,B00713458,HN_Set-5,HNSCC,NaN,NaN,SUCCESS,NaN,Not Detected,Not Detected,0.000000,NaN,NaN,16.12,4.0,0.0,06/11/2024,NaN,08/09/2024
303,MATRIX_HNSCC,FB12679954,A1086312,B00713459,HN_Set-3,HNSCC,NaN,NaN,SUCCESS,NaN,Detected,Detected,0.001033,HPV16,6.42,13.08,4.0,0.0,06/11/2024,NaN,08/09/2024
333,MATRIX_HNSCC,FB12778771,A1086313,B00713460,HN_Set-1,HNSCC,NaN,NaN,SUCCESS,NaN,Not Detected,Not Detected,0.000000,HPV16,18.89,10.69,4.0,0.0,06/11/2024,NaN,08/09/2024


# Update the ad-hoc report of SOW197 with pan-cancer binary caller result, TFv2 that generated by Evan 

In [30]:
# Readin the ad-hoc report (current using BIP LR binary call and TFv1)
df_AZ_inf_final = pd.read_csv(f"{work_dir}/df_AZ_inf_final.csv", header = 0)
df_AZ_inf_final.head()

,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Cancertype,Tumor_stage,Visit_name,Sample_status,Sample_comment,ctDNA_detected,Tumor_methylation_status,Tumor_methylation_score,Virus_strain,Virus_copy_number,cfDNA_ng_yield,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date
0,MATRIX_HNSCC,FB12680481,A1086304,B00713440,HN_Set-6,HNSCC,NaN,NaN,SUCCESS,NaN,Detected,Detected,0.001851,NaN,NaN,19.21,3.5,0.0,06/11/2024,NaN,08/09/2024
1,MATRIX_HNSCC,FB12680088,A1086305,B00713441,HN_Set-5,HNSCC,NaN,NaN,SUCCESS,NaN,Detected,Detected,0.006236,NaN,NaN,18.16,3.0,0.0,06/11/2024,NaN,08/09/2024
2,MATRIX_HNSCC,FB12679892,A1086306,B00713443,HN_Set-5,HNSCC,NaN,NaN,SUCCESS,NaN,Detected,Detected,0.016373,NaN,NaN,25.46,4.0,0.0,06/11/2024,NaN,08/09/2024
3,MATRIX_HNSCC,FB12679870,A1086307,B00713437,HN_Set-4,HNSCC,NaN,NaN,SUCCESS,NaN,Not Detected,Not Detected,0.000000,NaN,NaN,13.61,4.0,0.0,06/11/2024,NaN,08/09/2024
4,MATRIX_HNSCC,FB12778707,A1086308,B00713438,HN_Set-1,HNSCC,NaN,NaN,SUCCESS,NaN,Detected,Detected,0.002153,HPV16,161.54,6.77,4.0,0.0,06/11/2024,NaN,08/09/2024


In [31]:
# Readin Evan's udpated report 
df_AZ_updated_pan_tfv2_full_report = pd.read_csv(f"/home/ewarner/AZN/Infinity/AZN_SOW197_Infinity/AZN_SOW197_Infinity_01/tb/20240820_AstraZeneca_SOW197_30Samples_TFv2_InfinityReport.csv", header = 0)

# Subset the report
df_AZ_updated_pan_tfv2_full_report_sub = df_AZ_updated_pan_tfv2_full_report[['GHSampleID', 'Tumor_methylation_status', 'Tumor_methylation_score']]
df_AZ_updated_pan_tfv2_full_report_sub_dedup = df_AZ_updated_pan_tfv2_full_report_sub.drop_duplicates(subset = 'GHSampleID')

# Add new column "ctDNA_detected"
df_AZ_updated_pan_tfv2_full_report_sub_dedup['ctDNA_detected'] = df_AZ_updated_pan_tfv2_full_report_sub_dedup['Tumor_methylation_status']
df_AZ_updated_pan_tfv2_full_report_sub_dedup.head()

,GHSampleID,Tumor_methylation_status,Tumor_methylation_score,ctDNA_detected
0,B00713440,Detected,0.001915,Detected
64,B00713441,Detected,0.070738,Detected
138,B00713443,Detected,0.223826,Detected
197,B00713437,Not Detected,0.000059,Not Detected
214,B00713438,Detected,0.002366,Detected


In [19]:
# # Update the column values while the original column order is changed

# # Set 'GHSampleID' as the index for merging purpose
# df_AZ_inf_final.set_index('GHSampleID', inplace=True)
# df_AZ_updated_pan_tfv2_full_report_sub_dedup.set_index('GHSampleID', inplace=True)

# # Update the specified columns
# df_AZ_inf_final.update(df_AZ_updated_pan_tfv2_full_report_sub_dedup[['ctDNA_detected', 'Tumor_methylation_status', 'Tumor_methylation_score']])

# # Reset the index to bring back 'GHSampleID' as a column
# df_AZ_inf_final.reset_index(inplace=True)

# # Save the data
# df_AZ_inf_final.to_csv(f"{work_dir}/df_AZ_inf_final_08202024.csv", index = False)

# # Display the updated dataframe
# df_AZ_inf_final.head()

,GHSampleID,Study_ID,Customer_SampleID,GHRequestID,Patient_ID,Cancertype,Tumor_stage,Visit_name,Sample_status,Sample_comment,ctDNA_detected,Tumor_methylation_status,Tumor_methylation_score,Virus_strain,Virus_copy_number,cfDNA_ng_yield,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date
0,B00713440,MATRIX_HNSCC,FB12680481,A1086304,HN_Set-6,HNSCC,NaN,NaN,SUCCESS,NaN,Detected,Detected,0.001915,NaN,NaN,19.21,3.5,0.0,06/11/2024,NaN,08/09/2024
1,B00713441,MATRIX_HNSCC,FB12680088,A1086305,HN_Set-5,HNSCC,NaN,NaN,SUCCESS,NaN,Detected,Detected,0.070738,NaN,NaN,18.16,3.0,0.0,06/11/2024,NaN,08/09/2024
2,B00713443,MATRIX_HNSCC,FB12679892,A1086306,HN_Set-5,HNSCC,NaN,NaN,SUCCESS,NaN,Detected,Detected,0.223826,NaN,NaN,25.46,4.0,0.0,06/11/2024,NaN,08/09/2024
3,B00713437,MATRIX_HNSCC,FB12679870,A1086307,HN_Set-4,HNSCC,NaN,NaN,SUCCESS,NaN,Not Detected,Not Detected,0.000059,NaN,NaN,13.61,4.0,0.0,06/11/2024,NaN,08/09/2024
4,B00713438,MATRIX_HNSCC,FB12778707,A1086308,HN_Set-1,HNSCC,NaN,NaN,SUCCESS,NaN,Detected,Detected,0.002366,HPV16,161.54,6.77,4.0,0.0,06/11/2024,NaN,08/09/2024


In [39]:
# Update the column values while maintaining the original column order not changed

# Merge the updated values into df_AZ_inf_final based on 'GHSampleID'
df_merged = df_AZ_inf_final.merge(df_AZ_updated_pan_tfv2_full_report_sub_dedup[['GHSampleID', 'ctDNA_detected', 'Tumor_methylation_status', 'Tumor_methylation_score']],
                                  on='GHSampleID', 
                                  how='left', 
                                  suffixes=('', '_updated'))

# Update the original columns with the values from the merged columns if they are not NaN
df_merged['Tumor_methylation_status'] = df_merged['Tumor_methylation_status_updated'].combine_first(df_merged['Tumor_methylation_status'])
df_merged['Tumor_methylation_score'] = df_merged['Tumor_methylation_score_updated'].combine_first(df_merged['Tumor_methylation_score'])
df_merged['ctDNA_detected'] = df_merged['ctDNA_detected_updated'].combine_first(df_merged['ctDNA_detected'])


# Drop the temporary updated columns
df_merged.drop(columns=['Tumor_methylation_status_updated', 'Tumor_methylation_score_updated', 'ctDNA_detected_updated'], inplace=True)

# Ensure the dataframe maintains the original order and columns of df_AZ_inf_final
df_AZ_inf_final_new = df_merged[df_AZ_inf_final.columns]

# Updating 'Tumor_methylation_score' based on 'Tumor_methylation_status'
df_AZ_inf_final_new['Tumor_methylation_score'] = np.where(df_AZ_inf_final_new['Tumor_methylation_status'] == "Not Detected", 0, df_AZ_inf_final_new['Tumor_methylation_score'])

# Save the data
df_AZ_inf_final_new.to_csv(f"{work_dir}/df_AZ_inf_final_08202024.csv", index = False)

# Display the updated dataframe
df_AZ_inf_final_new


,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Cancertype,Tumor_stage,Visit_name,Sample_status,Sample_comment,ctDNA_detected,Tumor_methylation_status,Tumor_methylation_score,Virus_strain,Virus_copy_number,cfDNA_ng_yield,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date
0,MATRIX_HNSCC,FB12680481,A1086304,B00713440,HN_Set-6,HNSCC,NaN,NaN,SUCCESS,NaN,Detected,Detected,0.001915,NaN,NaN,19.21,3.5,0.0,06/11/2024,NaN,08/09/2024
1,MATRIX_HNSCC,FB12680088,A1086305,B00713441,HN_Set-5,HNSCC,NaN,NaN,SUCCESS,NaN,Detected,Detected,0.070738,NaN,NaN,18.16,3.0,0.0,06/11/2024,NaN,08/09/2024
2,MATRIX_HNSCC,FB12679892,A1086306,B00713443,HN_Set-5,HNSCC,NaN,NaN,SUCCESS,NaN,Detected,Detected,0.223826,NaN,NaN,25.46,4.0,0.0,06/11/2024,NaN,08/09/2024
3,MATRIX_HNSCC,FB12679870,A1086307,B00713437,HN_Set-4,HNSCC,NaN,NaN,SUCCESS,NaN,Not Detected,Not Detected,0.000000,NaN,NaN,13.61,4.0,0.0,06/11/2024,NaN,08/09/2024
4,MATRIX_HNSCC,FB12778707,A1086308,B00713438,HN_Set-1,HNSCC,NaN,NaN,SUCCESS,NaN,Detected,Detected,0.002366,HPV16,161.54,6.77,4.0,0.0,06/11/2024,NaN,08/09/2024
5,MATRIX_HNSCC,FB12680381,A1086309,B00713439,HN_Set-4,HNSCC,NaN,NaN,SUCCESS,NaN,Not Detected,Not Detected,0.000000,HPV16,13.38,11.27,3.5,0.0,06/11/2024,NaN,08/09/2024
6,MATRIX_HNSCC,FB12679956,A1086310,B00713455,HN_Set-4,HNSCC,NaN,NaN,SUCCESS,NaN,Detected,Detected,0.000312,HPV16,92.81,10.92,3.5,0.0,06/11/2024,NaN,08/09/2024
7,MATRIX_HNSCC,FB12679876,A1086311,B00713458,HN_Set-5,HNSCC,NaN,NaN,SUCCESS,NaN,Not Detected,Not Detected,0.000000,NaN,NaN,16.12,4.0,0.0,06/11/2024,NaN,08/09/2024
8,MATRIX_HNSCC,FB12679954,A1086312,B00713459,HN_Set-3,HNSCC,NaN,NaN,SUCCESS,NaN,Not Detected,Not Detected,0.000000,HPV16,6.42,13.08,4.0,0.0,06/11/2024,NaN,08/09/2024
9,MATRIX_HNSCC,FB12778771,A1086313,B00713460,HN_Set-1,HNSCC,NaN,NaN,SUCCESS,NaN,Not Detected,Not Detected,0.000000,HPV16,18.89,10.69,4.0,0.0,06/11/2024,NaN,08/09/2024


In [40]:
df_AZ_inf_final

,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Cancertype,Tumor_stage,Visit_name,Sample_status,Sample_comment,ctDNA_detected,Tumor_methylation_status,Tumor_methylation_score,Virus_strain,Virus_copy_number,cfDNA_ng_yield,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date
0,MATRIX_HNSCC,FB12680481,A1086304,B00713440,HN_Set-6,HNSCC,NaN,NaN,SUCCESS,NaN,Detected,Detected,0.001851,NaN,NaN,19.21,3.5,0.0,06/11/2024,NaN,08/09/2024
1,MATRIX_HNSCC,FB12680088,A1086305,B00713441,HN_Set-5,HNSCC,NaN,NaN,SUCCESS,NaN,Detected,Detected,0.006236,NaN,NaN,18.16,3.0,0.0,06/11/2024,NaN,08/09/2024
2,MATRIX_HNSCC,FB12679892,A1086306,B00713443,HN_Set-5,HNSCC,NaN,NaN,SUCCESS,NaN,Detected,Detected,0.016373,NaN,NaN,25.46,4.0,0.0,06/11/2024,NaN,08/09/2024
3,MATRIX_HNSCC,FB12679870,A1086307,B00713437,HN_Set-4,HNSCC,NaN,NaN,SUCCESS,NaN,Not Detected,Not Detected,0.000000,NaN,NaN,13.61,4.0,0.0,06/11/2024,NaN,08/09/2024
4,MATRIX_HNSCC,FB12778707,A1086308,B00713438,HN_Set-1,HNSCC,NaN,NaN,SUCCESS,NaN,Detected,Detected,0.002153,HPV16,161.54,6.77,4.0,0.0,06/11/2024,NaN,08/09/2024
5,MATRIX_HNSCC,FB12680381,A1086309,B00713439,HN_Set-4,HNSCC,NaN,NaN,SUCCESS,NaN,Not Detected,Not Detected,0.000000,HPV16,13.38,11.27,3.5,0.0,06/11/2024,NaN,08/09/2024
6,MATRIX_HNSCC,FB12679956,A1086310,B00713455,HN_Set-4,HNSCC,NaN,NaN,SUCCESS,NaN,Not Detected,Not Detected,0.000000,HPV16,92.81,10.92,3.5,0.0,06/11/2024,NaN,08/09/2024
7,MATRIX_HNSCC,FB12679876,A1086311,B00713458,HN_Set-5,HNSCC,NaN,NaN,SUCCESS,NaN,Not Detected,Not Detected,0.000000,NaN,NaN,16.12,4.0,0.0,06/11/2024,NaN,08/09/2024
8,MATRIX_HNSCC,FB12679954,A1086312,B00713459,HN_Set-3,HNSCC,NaN,NaN,SUCCESS,NaN,Detected,Detected,0.001033,HPV16,6.42,13.08,4.0,0.0,06/11/2024,NaN,08/09/2024
9,MATRIX_HNSCC,FB12778771,A1086313,B00713460,HN_Set-1,HNSCC,NaN,NaN,SUCCESS,NaN,Not Detected,Not Detected,0.000000,HPV16,18.89,10.69,4.0,0.0,06/11/2024,NaN,08/09/2024
